# Tarea 1 - Computación Cienctífica y Ciencia de los Datos
## Vectorización en Python: Generalización de autómatas de Conway
### Vicente Mieres.

## 1. Introducción

En el presente informe, se detalla la implmentacion de un generalización del juego de la vida de Conway, matriz infita con valores 0 o 1 por celda, las cuales "evolucionan" con el paso del tiempo. En particular para este trabajo, se debe extrapolar a grillas de n-dimensiones, con m posibles valores por celda. Además, este debe ser eficiente, por lo que la utilización de librerías como numpy y, sobretodo, cupy, son un requisito fundamental.

De esta forma, el objetivo de esta entrega, corresponde a la implementacion *eficiente* de un simulador, evitando el uso de ciclos tradicionales (ciclos *for* explícitos) y utilizando vectorización para trabajar de forma paralela. Además, se implmentará la version iterativa del problema, esto con el fin de realizar una comparativa a nivel de tiempo de ejecución, y asi demostrar la eficiencia de trabajar con programación paralela. Finalmente, se presentan diferentes ejemplos de la ejecución del código, para distintas combinaciones de parámetros.

## 2. Desarrollo

A continuación se detallan las consideraciones para poder implementar el autómata generalizado, tanto en su versión iterativa como en la optimizada.

### 2.1 Librerías

Para llevar a cabo el objetivo planteado, es necesario utilizar diferentes librerías, estas para poder manejar arreglos grandes, manejar la aceleracion por GPU requerida, realizar operaciones vectoriales, entre otros. Esta  se prensentan en el siguiente listado.

- **NumPy**: Encargada de la creación y manipulación de arreglos multidimensionales, operaciones vectorizadas, entre otros.
- **CuPy**: Implementacion de NumPy compatible con GPU. Permite acelerar el calculo de arreglos grandes utilizando computación paralela.
- **matplotlib.pyplot**: Biblioteca de visualización. Creación de graficas.
- **SciPy.ndimage.convolve**: Función específica de SciPy para el procesamiento de imagenes (Explicación más adelante).
- **matplotlib.use('TkAgg')**: Configura el backend de Matplotlib para creación de ventanas externas e interacción con gráficas.

In [3]:
import numpy as np
# import cupy as cp
import matplotlib.pyplot as plt
from scipy.ndimage import convolve

import matplotlib
matplotlib.use('TkAgg')  # Usar TkAgg backend para ventanas externas

### 2.2 Versión iterativa de la solución

Con el objetivo de poder comparar tiempos de ejecución, es necesario desarrollar una implementación estándar del autámata utilizando metodos iterativos. Es decir, la solución inicial planteada no emplea técnias de vectorización, recurriendo exclusivamente al uso de ciclos *for* anidados para realizar los calculos a lo largo de la grilla y sus **n** dimensiones.

AGREGAR LO DE LOS PARAMETROS

### 2.3 Versión eficiente de la solución

Una vez entendido el problema, y tomando como base la implemtación anterior. Se presenta a continuación la versión eficiente de la solución, es decir utilizando operaciones vectoriales para optimizar el manejo de arreglos y así obtener mejores tiempos de ejecución. En particular, la idea generalizada de esta implementación toma como base la *convolución*, de tal manera que se debe crear un kernel que será aplicado a todos los elementos de un grilla, y así poder obtener la suma de los vecinos requerida para posteriormente evolucionar la grilla según las reglas establecidas.

De esta forma, la solucion escogida esta conformada por diferentes parámetros y componentes. En cuanto a los parámetros, estos son los exactamente los mismos, puesto que de otra forma, se obtendrían resultados diferentes, lo cual no es optimo ni esperado. Luego, los diferentes componentes, o tambien llamados pasos, de la solución corresponden a todas las operaciones que debe hacer el simulador. Estos se encuentran listados a continuación.

- Creación de un kernel euclideano (fijo durante toda la simulación)
- Expandir la grilla
- Obtener la suma de vecinos
- Calcular intervalos
- Evolucionar la grilla


Con esto en consideración, se presenta la explicacion de cada una de los componentes implicados.

#### 2.3.1 Creación del Kernel Euclideano

En primer lugar, se considera la funcion **euclidean_kernel**, esta cumple con el objetivo de crear un máscara que recorrerá la grilla, aplicando cierto criterio, en este caso en particular, un kernel euclideano, es decir que considera a todos los vecinos de una celda en un radio r dado.

In [4]:
def euclidean_kernel(dimensions, radius):
    """
    Build an euclidean kernel. Circle shape in 2D, spheric shape in 3D, etc.
    
    Parámetros:
    - dimensions: Number of dimensions.
    - radius: Radius of neighbours.
    
    Return:
    - kernel: Numpy array of 1s where there is neighbourhood and 0s where there's not.
    """
    # Kernel dimension
    kernel_shape = (2 * radius + 1,) * dimensions
    kernel_size = np.prod(kernel_shape)
    
    # Create an array with kernels shape
    linear_indices = np.arange(kernel_size).reshape(kernel_shape)
    
    # Obtain centers
    center_coords = np.array([radius] * dimensions)
    
    # Transform indices into coordinates
    coords = np.unravel_index(linear_indices.flatten(), kernel_shape)
    coords = np.array(coords).reshape(dimensions, -1).T
    
    # Obtain distances using the euclidean function
    squared_distances = np.sum((coords - center_coords)**2, axis=1)
    kernel = (squared_distances <= radius**2).astype(np.int32)
    kernel = kernel.reshape(kernel_shape)
    
    # Excludes central cell
    center_index = tuple(radius for _ in range(dimensions))
    kernel[center_index] = 0
    
    return kernel

#### 2.3.2 Expansión de la grilla

El siguiente paso a considerar, es que tal como indica el enunciado, se requiere que la grilla sea una toroide, es decir que para aquellas celdas que se encuentran en las orillas, sea posible realizar la suma de vecino de forma correcta. Para esto se utiliza la funcion *pad* de numpy con el modo *wrap*, la cual permite simular el toroide.

In [5]:
def expand_grid(grid, radius):
    """
    Expands the grid with periodic boundaries (toroid).
    
    Parameters:
    - grid: NumPy ndarray.
    - radius: Neighborhood radius that determines how much to expand.
    
    Returns:
    - expanded_grid: Grid with periodic boundaries.
    """
    return np.pad(grid, radius, mode='wrap')

#### 2.3.3 Obtención de suma de vecinos

Tal como se mecionó anteriormente, para cada celda se debe calcular la suma de los vecinos, sin considerar la celda misma. Para esto se hace uso de la funcion *convolve* de forma similar a como se utiliza en procesamiento de imágenes. Un kernel (euclideano en este caso) es aplicado a toda la grilla, obteniendo así, una matríz nueva que contiene solamente la suma de vecinos para esa celda en particular. Además es necesario considerar, que al realizar esta convolución, el tamaño original de la grilla cambia, en este caso se hace mas grande, por tanto es necesario extrar la parte valida.

In [26]:
def get_neighbor_sum(expanded_grid, kernel):
    """
    Calculates the sum of values in the neighborhood for each cell.
    
    Parameters:
    - expanded_grid: Grid with expanded borders.
    - kernel: Kernel that defines the neighborhood.
    
    Returns:
    - neighbor_sum: Array with the sum of neighboring values for each cell.
    """
    # Convolution to obtain the sum of neighbors
    neighbor_sum = convolve(expanded_grid, kernel, mode='wrap')
    # Extract the valid part of the convolution
    radius = kernel.shape[0] // 2
    slices = tuple(slice(radius, -radius) for _ in range(expanded_grid.ndim))
    neighbor_sum = neighbor_sum[slices]
    
    return neighbor_sum

#### 2.3.4 Obtención de intervalos

Para este caso, simplemente es necesario utilizar la función *linspace* de numpy para generar los tres intervalos necesarios, con valor máximo SM, valor que corresponde a la suma de los vecinos multiplicado por el parámetro m.

In [7]:
def get_intervals(kernel, m):
   """
   Calculates the intervals for applying the evolution rules.
   
   Parameters:
   - kernel: Kernel that defines the neighborhood.
   - m: Maximum number of states per cell.
   
   Returns:
   - intervals: Array with the limits of the 3 intervals.
   """
   num_neighbors = np.sum(kernel)
   SM = m * num_neighbors
   intervals = np.linspace(0, SM, 4)  # 3 intervals
   return intervals

#### 2.3.5 Evolución de la grilla

La siguiente función, es la encargada de aplicar las reglas establecidas anteriormente. Primero, creando máscaras booleanas, que cumplan con los intervalos. Y luego utiliza la indexación de las grillas para sumar o restar 1 según corresponda.

In [8]:
def evolve_grid(grid, neighbor_sum, intervals, m):
   """
   Applies evolution rules to the cellular automaton.
   
   Parameters:
   - grid: Current configuration of the automaton.
   - neighbor_sum: Sum of values in the neighborhood for each cell.
   - intervals: Limits of the intervals for applying rules.
   - m: Maximum number of states per cell.
   
   Returns:
   - new_grid: Grid evolved according to the rules.
   """
   # Copy of the grid to avoid modifying the original
   new_grid = grid.copy()
   
   # Apply rules with masks
   mask1 = (neighbor_sum >= intervals[0]) & (neighbor_sum < intervals[1])
   mask2 = (neighbor_sum >= intervals[1]) & (neighbor_sum < intervals[2])
   mask3 = (neighbor_sum >= intervals[2])
   
   new_grid[mask1] = np.maximum(new_grid[mask1] - 1, 0)  
   new_grid[mask2] = np.minimum(new_grid[mask2] + 1, m)  
   new_grid[mask3] = np.maximum(new_grid[mask3] - 1, 0)
   return new_grid

## 3. Funciones Auxiliares

A continuación se presentan funciones auxiliares, estas  se utilizan para creación de grillas con patrones específicos, o visualización de la mismas para N dimensiones.

In [9]:
def create_grid(d, grid_size, m, pattern='random'):
    """
    Create a n-dimensional grid of a specified pattern.
   
    Parameters:
    - d: number of dimensions
    - grid_size: size of each dimension. (Array lenght)
    - m: max value for each cell
    - pattern: type of initial pattern ('random', 'central', 'checkerboard', 'cross', 'line')

    Return:
    - grid: n-dimensional grid
    """
    
    if isinstance(grid_size, int):
        grid_size = tuple([grid_size] * d)
    else:
        grid_size = tuple(grid_size)
   
    if pattern == 'random':
        # Grilla con valores aleatorios entre 0 y m
        return np.random.randint(0, m+1, grid_size)
   
    elif pattern == 'central':
        # Grilla con valor alto en el centro y ceros alrededor
        grid = np.zeros(grid_size, dtype=int)
        center = tuple(s // 2 for s in grid_size)
       
        # Colocar valor m en el centro
        indices = tuple(slice(c-1, c+2) for c in center)
        grid[indices] = m
        return grid
   
    elif pattern == 'checkerboard':
        # Patrón de tablero de ajedrez
        indices = np.indices(grid_size)
        sum_indices = np.sum(indices, axis=0)
        return (sum_indices % 2) * m
        
    elif pattern == 'cross':
        # Patrón de cruz
        grid = np.zeros(grid_size, dtype=int)
        center = tuple(s // 2 for s in grid_size)
        
        # Para cada dimensión, crear una línea a través del centro
        for dim in range(d):
            # Crear índices para la línea en esta dimensión
            indices = [slice(0, s) if i != dim else center[i] for i, s in enumerate(grid_size)]
            grid[tuple(indices)] = m
        
        return grid
        
    elif pattern == 'line':
        # Patrón de línea horizontal 
        grid = np.zeros(grid_size, dtype=int)
    
        # Para una línea horizontal, necesitamos fijar la coordenada y y variar x
        if d >= 2:
            indices = [grid_size[1] // 2, slice(0, grid_size[0])]
            # Para dimensiones adicionales, elegimos el punto central
            for dim in range(2, d):
                indices.append(grid_size[dim] // 2)
            grid[tuple(indices)] = m
        else:  # Para el caso unidimensional
            grid[:] = m
        
        return grid
   
    else:
        # Patrón predeterminado: valores 0
        return np.zeros(grid_size, dtype=int)

In [10]:
def visualize(grid, m, r, iterations=5, pause=1, cmap='viridis'):
    """
    Simulate and visualize the evolution of the cellular automaton in sequential windows.
   
    Parámetros:
    - grid: 2D initial grid
    - m: max value for each cell
    - r: neighborhood radius
    - iterations: Number of iterations
    - pause: time between iterations (seconds)
    - cmap: color map
    """

    # Deactivate interactive mode
    plt.ioff()
    actual_grid = grid.copy()
   
    # history of generated grids
    history = [actual_grid.copy()]

    kernel = euclidean_kernel(grid.ndim, r)
   
   # Simulation
    for step in range(iterations):
        expanded_grid = expand_grid(actual_grid, r)
        neighbor_sum = get_neighbor_sum(expanded_grid, kernel)
        intervals = get_intervals(kernel, m)
        actual_grid = evolve_grid(actual_grid, neighbor_sum, intervals, m)
        history.append(actual_grid.copy())
   
    # show every grid
    for step, grid in enumerate(history):
        fig = plt.figure(figsize=(8, 6), num=f"Cellular Automaton - Step {step}")
        plt.imshow(grid, cmap=cmap, interpolation='nearest')
        plt.colorbar(label='Cell State')
        plt.title(f"Step {step}" + (" (Initial)" if step == 0 else ""))
        plt.grid(True, which='both', color='white', linestyle='-', alpha=0.3)
       
        # Every 1 second (default) shows an image of the grid.
        if step < len(history) - 1:
            plt.tight_layout()
            plt.show(block=False)
            plt.pause(pause)
            plt.close()
        else:
            plt.tight_layout()
            plt.show()

In [ ]:
def visualize_nd(initial_grid, m, r, iterations=5, pause=1, cmap='viridis', visualization_axes=(0, 1), fixed_values=None):
    """
    Simulate and visualize the evolution of the cellular automaton in higher dimensions.
    Shows 2D slices keeping the other dimensions fixed.
    
    Parameters:
    - initial_grid: n-dimensional initial grid
    - m: Maximum state value
    - r: Neighborhood radius
    - iterations: Number of iterations to simulate
    - pause: Time between iterations (seconds)
    - cmap: Color map
    - visualization_axes: Tuple with the two axes to visualize (default (0,1))
    - fixed_values: Dictionary with fixed values for other dimensions {dim: value}
                   If None, the midpoint of each non-visualized dimension will be used
    
    """
    
    # Deactivate interactive mode
    plt.ioff()
    
    # Start simulation
    current_grid = initial_grid.copy()
    
    # history of generated grid
    history = [current_grid.copy()]
    
    kernel = euclidean_kernel(initial_grid.ndim, r)
    
    # Simulation
    for step in range(iterations):
        expanded_grid = expand_grid(current_grid, r)
        neighbor_sum = get_neighbor_sum(expanded_grid, kernel)
        intervals = get_intervals(kernel, m)
        current_grid = evolve_grid(current_grid, neighbor_sum, intervals, m)
        history.append(current_grid.copy())
    
    # Determine fixed values for non-visualized dimensions
    if fixed_values is None:
        fixed_values = {}
        
    # Get grid dimensions
    dimensions = initial_grid.shape
    ndim = len(dimensions)
    
    # Verify that visualization_axes are valid
    axis_x, axis_y = visualization_axes
    if axis_x >= ndim or axis_y >= ndim or axis_x < 0 or axis_y < 0 or axis_x == axis_y:
        raise ValueError(f"Visualization axes must be different and less than {ndim}")
    
    # Set default values for non-visualized dimensions
    for dim in range(ndim):
        if dim != axis_x and dim != axis_y and dim not in fixed_values:
            fixed_values[dim] = dimensions[dim] // 2  # Default midpoint
    
    # Show each grid sequentially
    for step, grid in enumerate(history):
        fig = plt.figure(figsize=(8, 6), num=f"Cellular Automaton {ndim}D - Step {step}")
        
        # Extract the 2D slice for visualization
        indices = [slice(None) if i == axis_x or i == axis_y else fixed_values[i] for i in range(ndim)]
        slice_2d = grid[tuple(indices)]
        
        # Show the 2D slice
        plt.imshow(slice_2d, cmap=cmap, interpolation='nearest')
        plt.colorbar(label='Cell State')
        
        # Create title with slice information
        slice_info = ", ".join([f"dim{i}={fixed_values[i]}" for i in range(ndim)
                               if i != axis_x and i != axis_y])
        plt.title(f"Step {step} ({slice_info})" + (" (Initial)" if step == 0 else ""))
        
        plt.xlabel(f"Dimension {axis_x}")
        plt.ylabel(f"Dimension {axis_y}")
        plt.grid(True, which='both', color='white', linestyle='-', alpha=0.3)
        
        if step < len(history) - 1:
            plt.tight_layout()
            plt.show(block=False)
            plt.pause(pause)
            plt.close()
        else:
            # For the last window, show without blocking
            plt.tight_layout()
            plt.show()

## 4. Ejemplos

In [59]:
r = 2
m = 1
griz_size = 10
d = 2
grid = create_grid(d, griz_size, m, pattern='cross')
visualize(grid, m,r, 10)

In [67]:
def get_value_toroidal(grid, coord, shape):
    wrapped = tuple((coord[i] % shape[i]) for i in range(len(coord)))
    subgrid = grid
    for idx in wrapped:
        subgrid = subgrid[idx]
    return subgrid

def generar_coordenadas_vecinas(coord, r):
    """
    Genera coordenadas vecinas dentro de una hiperesfera discreta de radio r.
    Excluye la celda central.
    """
    from itertools import product
    from math import sqrt

    d = len(coord)
    rangos = [range(c - r, c + r + 1) for c in coord]

    for offset in product(*rangos):
        # Calcular distancia euclidiana
        distancia = sqrt(sum((offset[i] - coord[i]) ** 2 for i in range(d)))
        if 0 < distancia <= r:
            yield offset


def inicializar_grilla_vacia(shape):
    if not shape:
        return 0
    return [inicializar_grilla_vacia(shape[1:]) for _ in range(shape[0])]

def asignar_valor(grilla, coord, valor):
    sub = grilla
    for i in range(len(coord) - 1):
        sub = sub[coord[i]]
    sub[coord[-1]] = valor

def recorrer_grilla(shape):
    from itertools import product
    return product(*[range(s) for s in shape])

def sumar_vecinos_iterativo(grid, r):
    """
    Calcula la suma de vecinos para cada celda de una grilla d-dimensional.
    """
    shape = []

    # Calcular shape desde grid
    sub = grid
    while isinstance(sub, list) and len(sub) > 0:
        shape.append(len(sub))
        sub = sub[0]

    nueva_grilla = inicializar_grilla_vacia(shape)

    for coord in recorrer_grilla(shape):
        suma = 0
        for vecina in generar_coordenadas_vecinas(coord, r):
            suma += get_value_toroidal(grid, vecina, shape)
        asignar_valor(nueva_grilla, coord, suma)

    return nueva_grilla


def aplicar_regla(grid, suma_vecinos, m, r):
    """
    Aplica la regla de evolución al autómata celular generalizado.

    Parámetros:
    - grid: grilla original (listas anidadas)
    - suma_vecinos: grilla con la suma de vecinos (listas anidadas)
    - m: máximo valor permitido por celda
    - r: radio de vecindad

    Retorna:
    - nueva_grilla: grilla actualizada tras aplicar la regla
    """
    from math import sqrt

    # Obtener shape
    shape = []
    sub = grid
    while isinstance(sub, list) and len(sub) > 0:
        shape.append(len(sub))
        sub = sub[0]

    # Contar vecinos dentro de la esfera discreta (sin contar el centro)
    from itertools import product

    d = len(shape)
    total_vecinos = 0
    for delta in product(*[range(-r, r+1) for _ in range(d)]):
        if all(x == 0 for x in delta):
            continue
        distancia = sqrt(sum(x**2 for x in delta))
        if distancia <= r:
            total_vecinos += 1

    SM = m * total_vecinos
    tercio = SM / 3

    nueva_grilla = inicializar_grilla_vacia(shape)

    for coord in recorrer_grilla(shape):
        valor_actual = get_value_toroidal(grid, coord, shape)
        suma = get_value_toroidal(suma_vecinos, coord, shape)

        if suma < tercio:
            nuevo = max(0, valor_actual - 1)
        elif suma < 2 * tercio:
            nuevo = min(m, valor_actual + 1)
        else:
            nuevo = max(0, valor_actual - 1)

        asignar_valor(nueva_grilla, coord, nuevo)

    return nueva_grilla

def paso_automata(grid, m, r):
    """
    Realiza un paso completo del autómata celular:
    - Calcula suma de vecinos
    - Aplica la regla de evolución
    """
    suma = sumar_vecinos_iterativo(grid, r)
    nueva = aplicar_regla(grid, suma, m, r)
    return nueva

def simular_automata(grid, m, r, n_pasos, mostrar=True, delay=0.8):
    """
    Simula el autómata celular por múltiples pasos y grafica si es 2D.

    Parámetros:
    - grid: grilla inicial como listas anidadas
    - m: máximo valor por celda
    - r: radio de vecindad
    - n_pasos: pasos de simulación
    - mostrar: si True, muestra la grilla paso a paso
    - delay: segundos entre gráficos (si mostrar es True)

    Retorna:
    - historia: lista de grillas generadas
    """
    import matplotlib.pyplot as plt
    import time

    historia = [grid]

    for paso in range(1, n_pasos + 1):
        grid = paso_automata(grid, m, r)
        historia.append(grid)

        if mostrar:
            print(f"Paso {paso}")
            if len(grid) > 0 and isinstance(grid[0], list) and isinstance(grid[0][0], int):
                # Grilla 2D → graficamos
                plt.imshow(grid, cmap='viridis', vmin=0, vmax=m)
                plt.title(f"Paso {paso}")
                plt.colorbar()
                plt.pause(delay)
                plt.clf()  # Limpia figura para el siguiente paso
            else:
                # Otras dimensiones → solo texto
                print("Vista no compatible con más de 2 dimensiones")
                imprimir_grilla(grid)

    return historia


def imprimir_grilla(grilla):
    for fila in grilla:
        print(fila)


In [80]:
# version optimizada
r = 2
m = 2
griz_size = 7
d = 2
grid = create_grid(d, griz_size, m, pattern='cross')
visualize(grid, m,r, 10)

In [82]:
r = 2
m = 2
griz_size = 7
d = 2

grid = create_grid(d, griz_size, m, pattern='cross')
print(grid)

historia = simular_automata(grid.tolist(), m=2, r=2, n_pasos=1, mostrar=True)    


[[0 0 0 2 0 0 0]
 [0 0 0 2 0 0 0]
 [0 0 0 2 0 0 0]
 [2 2 2 2 2 2 2]
 [0 0 0 2 0 0 0]
 [0 0 0 2 0 0 0]
 [0 0 0 2 0 0 0]]
Paso 1


In [ ]:


# Llamamos a la función sumar_vecinos_iterativo
vecinos = sumar_vecinos_iterativo(grid.tolist(), r)

# Función simple para imprimir grilla


print("Grilla original:")
imprimir_grilla(grid)

print("\nSuma de vecinos (radio euclidiano 1):")
imprimir_grilla(vecinos)


Grilla original:
[[0 0 1 0 0]
 [0 0 1 0 0]
 [1 1 1 1 1]
 [0 0 1 0 0]
 [0 0 1 0 0]]
[[0 0 1 0 0]
 [0 0 1 0 0]
 [1 1 1 1 1]
 [0 0 1 0 0]
 [0 0 1 0 0]]
[[1 1 1 1 1]
 [1 1 1 1 1]
 [1 1 1 1 1]
 [1 1 1 1 1]
 [1 1 1 1 1]]
[[0 0 1 0 0]
 [0 0 1 0 0]
 [1 1 1 1 1]
 [0 0 1 0 0]
 [0 0 1 0 0]]
[[0 0 1 0 0]
 [0 0 1 0 0]
 [1 1 1 1 1]
 [0 0 1 0 0]
 [0 0 1 0 0]]

Suma de vecinos (radio euclidiano 1):
[[3, 11, 12, 11, 3], [11, 16, 18, 16, 11], [12, 18, 20, 18, 12], [11, 16, 18, 16, 11], [3, 11, 12, 11, 3]]
[[11, 16, 18, 16, 11], [16, 19, 22, 19, 16], [18, 22, 24, 22, 18], [16, 19, 22, 19, 16], [11, 16, 18, 16, 11]]
[[12, 18, 20, 18, 12], [18, 22, 24, 22, 18], [20, 24, 24, 24, 20], [18, 22, 24, 22, 18], [12, 18, 20, 18, 12]]
[[11, 16, 18, 16, 11], [16, 19, 22, 19, 16], [18, 22, 24, 22, 18], [16, 19, 22, 19, 16], [11, 16, 18, 16, 11]]
[[3, 11, 12, 11, 3], [11, 16, 18, 16, 11], [12, 18, 20, 18, 12], [11, 16, 18, 16, 11], [3, 11, 12, 11, 3]]


In [45]:
kernel = euclidean_kernel(grid.ndim, r)
    
expanded_grid = expand_grid(grid, r)
neighbor_sum = get_neighbor_sum(expanded_grid, kernel)
neighbor_sum

array([[[ 3, 11, 12, 11,  3],
        [11, 16, 18, 16, 11],
        [12, 18, 20, 18, 12],
        [11, 16, 18, 16, 11],
        [ 3, 11, 12, 11,  3]],

       [[11, 16, 18, 16, 11],
        [16, 19, 22, 19, 16],
        [18, 22, 24, 22, 18],
        [16, 19, 22, 19, 16],
        [11, 16, 18, 16, 11]],

       [[12, 18, 20, 18, 12],
        [18, 22, 24, 22, 18],
        [20, 24, 24, 24, 20],
        [18, 22, 24, 22, 18],
        [12, 18, 20, 18, 12]],

       [[11, 16, 18, 16, 11],
        [16, 19, 22, 19, 16],
        [18, 22, 24, 22, 18],
        [16, 19, 22, 19, 16],
        [11, 16, 18, 16, 11]],

       [[ 3, 11, 12, 11,  3],
        [11, 16, 18, 16, 11],
        [12, 18, 20, 18, 12],
        [11, 16, 18, 16, 11],
        [ 3, 11, 12, 11,  3]]])

In [50]:
# Crear una grilla 2D con patrón 'cross'
np_grid = create_grid(d=2, grid_size=5, m=3, pattern='cross')
# prtin(np_grid)

# Simular 5 pasos con radio 1
historia = simular_automata(grid.tolist(), m=3, r=1, n_pasos=5)



Paso 1:
[[0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 2, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 2, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0]]
[[0, 0, 2, 0, 0], [0, 0, 2, 0, 0], [2, 2, 2, 2, 2], [0, 0, 2, 0, 0], [0, 0, 2, 0, 0]]
[[0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 2, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 2, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0]]

Paso 2:
[[0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0]]
[[0, 0, 1, 0, 0], [0, 0, 1, 0, 0], [1, 1, 1, 1, 1], [0, 0, 1, 0, 0], [0, 0, 1, 0, 0]]
[[0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0]]

Paso 3:
[[0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0], [0, 0, 0,